In [79]:
import pandas as pd

df = pd.read_excel("./input/rcr.xlsx", skiprows=12)
print(df.shape)
dfs = [df]

(201049, 16)


In [80]:
import os
import re

for fp in os.listdir("./historic/"):
    df = pd.read_csv(f"./historic/{fp}")
    print(fp, df.shape)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

2020-07-11_to_2021-01-02 (316394, 16)
2021-01-09_to_2021-07-03 (280890, 16)
2021-07-10_to_2022-01-01 (298564, 16)
2022-01-08_to_2022-07-02 (294187, 16)
2022-07-09_to_2022-12-31 (309979, 16)


In [81]:
df

,Country,Sub Family,Series,Model Description,Storage Capacity,CPU Group,CPU Ram,Screen Size Numeric,Gaming Products,PCHeight(mm),OS Ecosystem,Sub Form Factor,Price Band USD 3,Week End,SALES UNITS,SALES VALUE USD
0,France,133 Flip 13,ARCHOS,133 FLIP,32,ATOM Z3735F,2048,13.3,NO GAMING,20.0,Windows,Convertible,300-500,2/18/2023,1,308.954
1,France,Acer One 10,ACER ASPIRE ONE,ASPIRE ONE 10 S1002-16N3,32,ATOM Z3735G,1024,10.1,NO GAMING,178.5,Windows,Detachable,0-300,3/18/2023,1,251.942
2,France,Acer One 10,ACER ASPIRE ONE,ASPIRE ONE 10 S1002-19JN,32,ATOM Z3735F,2048,10.1,NO GAMING,178.5,Windows,Detachable,0-300,1/7/2023,2,212.370
3,France,Akoya E11201,MEDION AKOYA,AKOYA E11201,64,CELERON N,4096,11.6,NO GAMING,23.8,Windows,Traditional Notebook,0-300,1/7/2023,1,265.463
4,France,Akoya E11201,MEDION AKOYA,AKOYA E11201,64,CELERON N,4096,11.6,NO GAMING,23.8,Windows,Traditional Notebook,0-300,1/14/2023,6,1007.696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1701058,United Kingdom,Undefined,ZOOSTORM STORMF,Not Provided,1120,CORE I3,8192,NaN,NO GAMING,435.5,Windows,Traditional Desktop,800+,8/20/2022,13,10429.175
1701059,United Kingdom,Undefined,ZOOSTORM STORMF,Not Provided,1120,CORE I3,8192,NaN,NO GAMING,435.5,Windows,Traditional Desktop,800+,9/3/2022,14,11230.719
1701060,United Kingdom,Undefined,ZOOSTORM STORMF,Not Provided,1120,CORE I3,8192,NaN,NO GAMING,435.5,Windows,Traditional Desktop,800+,12/10/2022,18,14444.207
1701061,United Kingdom,Undefined,ZOOSTORM STORMF,Not Provided,1120,CORE I3,8192,NaN,NO GAMING,435.5,Windows,Traditional Desktop,800+,12/17/2022,17,13640.480


In [82]:
df[df.index.duplicated()]

,Country,Sub Family,Series,Model Description,Storage Capacity,CPU Group,CPU Ram,Screen Size Numeric,Gaming Products,PCHeight(mm),OS Ecosystem,Sub Form Factor,Price Band USD 3,Week End,SALES UNITS,SALES VALUE USD


In [83]:
# df["CLASS"] = ""
# df["Form Factor"] = ""
# uk = df[df.Country=="United Kingdom"]
# uk_apple_surface = uk[uk["Sub Family"].isin(["Apple Other", "Surface Pro", "Surface Laptop"])]
# uk_other = uk[~uk["Sub Family"].isin(["Apple Other", "Surface Pro", "Surface Laptop"])]

# figs = df[df.Country!="United Kingdom"]
# figs_windows = figs[figs["OS Ecosystem"]=="Windows"]

In [84]:
# df["CLASS"] = df["Sub Family"]
# df.loc[(df["OS Ecosystem"]=="Windows") & (~df.CLASS.str.contains("Surface")), "CLASS"] = "Windows OEM"
# df.loc[df.CLASS.str.contains("Chromebook"), "CLASS"] = "Chromebook"

In [85]:
df.loc[(df.Country!="United Kingdom") & (df["OS Ecosystem"]=="Windows") & (df["Sub Family"].str.contains("Surface")), "CLASS"] = df["Sub Family"]
df.loc[(df.Country!="United Kingdom") & (df["OS Ecosystem"]=="Windows") & (~df["Sub Family"].str.contains("Surface")), "CLASS"] = "Windows OEM"
df.loc[(df.Country!="United Kingdom") & (df["OS Ecosystem"]=="Apple"), "CLASS"] = df["Sub Family"]
df.loc[(df.Country!="United Kingdom") & (df["OS Ecosystem"]=="Google") & df["Sub Family"].str.contains("Chromebook"), "CLASS"] = "Chromebook"
df.loc[(df.Country!="United Kingdom") & (df["OS Ecosystem"]=="Google") & (~df["Sub Family"].str.contains("Chromebook", na=False)), "CLASS"] = "Other"
df.loc[(df.Country=="United Kingdom") & (df["OS Ecosystem"]=="Windows") & (df["Sub Family"]=="Undefined"), "CLASS"] = "Windows OEM"
df.loc[(df.Country=="United Kingdom") & (df["OS Ecosystem"]!="Windows") & (df["Sub Family"]=="Undefined"), "CLASS"] = "Other"

In [86]:
figs = df[df.Country!="United Kingdom"]
uk = df[df.Country=="United Kingdom"]
uk_apple_surface = uk[uk["Sub Family"].isin(["Apple Other", "Surface Pro", "Surface Laptop"])]
uk_other = uk[~uk["Sub Family"].isin(["Apple Other", "Surface Pro", "Surface Laptop"])]

In [87]:
lookup = pd.read_csv("./lookup/UK Lookup Table.txt")
uk_apple_surface = uk_apple_surface.merge(right=lookup, how="left", on=["Series", "Screen Size Numeric", "PCHeight(mm)"], suffixes=["_old", None])
uk_apple_surface.drop(columns="CLASS_old", inplace=True)

In [88]:
df = pd.concat([figs, uk_apple_surface, uk_other])

In [89]:
lookup = pd.read_csv("./lookup/Date Table.txt")
df["Week End"] = pd.to_datetime(df["Week End"], format="%m/%d/%Y")
lookup["Week End"] = pd.to_datetime(lookup["Week End"], format="%d/%m/%Y %H:%M:%S")
df = df.merge(right=lookup, how="left", on="Week End")

In [90]:
form_factor_map = {
    "Detachable":           "2in1",
    "Traditional Desktop":  "Desktop",
    "Traditional Notebook": "Notebook",
    "Convertible":          "2in1",
    "Large Slate":          "2in1",
    "AIO":                  "Desktop",
    "Ultraslim Notebook":   "Notebook"
}
df.loc[df.CLASS=="Surface Laptop Studio", "Sub Form Factor"] = "Convertible"
df["Form Factor"] = df["Sub Form Factor"].map(form_factor_map)
df

,Country,Sub Family,Series,Model Description,Storage Capacity,CPU Group,CPU Ram,Screen Size Numeric,Gaming Products,PCHeight(mm),...,Sub Form Factor,Price Band USD 3,Week End,SALES UNITS,SALES VALUE USD,CLASS,FY,FYWK,MTH,Form Factor
0,France,133 Flip 13,ARCHOS,133 FLIP,32,ATOM Z3735F,2048,13.3,NO GAMING,20.0,...,Convertible,300-500,2023-02-18,1,308.954,Windows OEM,2023,33,February,2in1
1,France,Acer One 10,ACER ASPIRE ONE,ASPIRE ONE 10 S1002-16N3,32,ATOM Z3735G,1024,10.1,NO GAMING,178.5,...,Detachable,0-300,2023-03-18,1,251.942,Windows OEM,2023,37,March,2in1
2,France,Acer One 10,ACER ASPIRE ONE,ASPIRE ONE 10 S1002-19JN,32,ATOM Z3735F,2048,10.1,NO GAMING,178.5,...,Detachable,0-300,2023-01-07,2,212.370,Windows OEM,2023,27,January,2in1
3,France,Akoya E11201,MEDION AKOYA,AKOYA E11201,64,CELERON N,4096,11.6,NO GAMING,23.8,...,Traditional Notebook,0-300,2023-01-07,1,265.463,Windows OEM,2023,27,January,Notebook
4,France,Akoya E11201,MEDION AKOYA,AKOYA E11201,64,CELERON N,4096,11.6,NO GAMING,23.8,...,Traditional Notebook,0-300,2023-01-14,6,1007.696,Windows OEM,2023,28,January,Notebook
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1701058,United Kingdom,Undefined,ZOOSTORM STORMF,Not Provided,1120,CORE I3,8192,NaN,NO GAMING,435.5,...,Traditional Desktop,800+,2022-08-20,13,10429.175,Windows OEM,2023,7,August,Desktop
1701059,United Kingdom,Undefined,ZOOSTORM STORMF,Not Provided,1120,CORE I3,8192,NaN,NO GAMING,435.5,...,Traditional Desktop,800+,2022-09-03,14,11230.719,Windows OEM,2023,9,September,Desktop
1701060,United Kingdom,Undefined,ZOOSTORM STORMF,Not Provided,1120,CORE I3,8192,NaN,NO GAMING,435.5,...,Traditional Desktop,800+,2022-12-10,18,14444.207,Windows OEM,2023,23,December,Desktop
1701061,United Kingdom,Undefined,ZOOSTORM STORMF,Not Provided,1120,CORE I3,8192,NaN,NO GAMING,435.5,...,Traditional Desktop,800+,2022-12-17,17,13640.480,Windows OEM,2023,24,December,Desktop


In [91]:
import xlwings as xw

for fp in os.listdir():
    if re.fullmatch(r"GfK-Weekly-UFIGS-FY\d{2}-FYWK\d{2} CYWK\d{2}.xlsx", fp):
        break

fy = df.FY.max()
fywk = df[df.FY==fy].FYWK.max()
cywk = (fywk + 26) % 52

df = df[[
    "Country", "Sub Family", "Series", "Model Description", "Storage Capacity", "CPU Group",
    "CPU Ram", "Screen Size Numeric", "PCHeight(mm)", "OS Ecosystem", "Sub Form Factor",
    "Price Band USD 3", "Week End", "SALES UNITS", "SALES VALUE USD", "CLASS", "Form Factor",
    "FYWK", "FY"
]]
df = df[df["Week End"] >= "2022-01-08"]
print(df.shape)

wb = xw.Book(fp)
sheet = wb.sheets("Output_Table")
sheet.range("A:S").clear_contents()
sheet["A1"].options(index=False, header=True).value = df
wb.save(f"GfK-Weekly-UFIGS-FY{fy:01}-FYWK{fywk:01} CYWK{cywk:01}.xlsx")
# wb.close()

(805215, 19)


In [92]:
df


,Country,Sub Family,Series,Model Description,Storage Capacity,CPU Group,CPU Ram,Screen Size Numeric,PCHeight(mm),OS Ecosystem,Sub Form Factor,Price Band USD 3,Week End,SALES UNITS,SALES VALUE USD,CLASS,Form Factor,FYWK,FY
0,France,133 Flip 13,ARCHOS,133 FLIP,32,ATOM Z3735F,2048,13.3,20.0,Windows,Convertible,300-500,2023-02-18,1,308.954,Windows OEM,2in1,33,2023
1,France,Acer One 10,ACER ASPIRE ONE,ASPIRE ONE 10 S1002-16N3,32,ATOM Z3735G,1024,10.1,178.5,Windows,Detachable,0-300,2023-03-18,1,251.942,Windows OEM,2in1,37,2023
2,France,Acer One 10,ACER ASPIRE ONE,ASPIRE ONE 10 S1002-19JN,32,ATOM Z3735F,2048,10.1,178.5,Windows,Detachable,0-300,2023-01-07,2,212.370,Windows OEM,2in1,27,2023
3,France,Akoya E11201,MEDION AKOYA,AKOYA E11201,64,CELERON N,4096,11.6,23.8,Windows,Traditional Notebook,0-300,2023-01-07,1,265.463,Windows OEM,Notebook,27,2023
4,France,Akoya E11201,MEDION AKOYA,AKOYA E11201,64,CELERON N,4096,11.6,23.8,Windows,Traditional Notebook,0-300,2023-01-14,6,1007.696,Windows OEM,Notebook,28,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1701058,United Kingdom,Undefined,ZOOSTORM STORMF,Not Provided,1120,CORE I3,8192,NaN,435.5,Windows,Traditional Desktop,800+,2022-08-20,13,10429.175,Windows OEM,Desktop,7,2023
1701059,United Kingdom,Undefined,ZOOSTORM STORMF,Not Provided,1120,CORE I3,8192,NaN,435.5,Windows,Traditional Desktop,800+,2022-09-03,14,11230.719,Windows OEM,Desktop,9,2023
1701060,United Kingdom,Undefined,ZOOSTORM STORMF,Not Provided,1120,CORE I3,8192,NaN,435.5,Windows,Traditional Desktop,800+,2022-12-10,18,14444.207,Windows OEM,Desktop,23,2023
1701061,United Kingdom,Undefined,ZOOSTORM STORMF,Not Provided,1120,CORE I3,8192,NaN,435.5,Windows,Traditional Desktop,800+,2022-12-17,17,13640.480,Windows OEM,Desktop,24,2023


In [93]:
df["Week End"].max()


Timestamp('2023-05-06 00:00:00')